# BT4012 Without Text Analysis

### Setting up the environment

In [ ]:
# libraries importing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tensorflow as tf

In [ ]:
# setting up

# import from github repo
url = 'https://raw.githubusercontent.com/LordZhiHao/BT4012_Fraud_Analytics_Project/main/fake_job_postings.csv'

# read data
data = pd.read_csv(url)
df = data.copy()
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


### Handling Null Values

EDA plz do refer other scripts

Will dive straight into handling the null values and outliers

In [ ]:
# keep track of what columns to keep and drop
cols_to_keep = []
cols_to_drop = []

In [ ]:
# check for NA values
df.isna().any()

job_id                 False
title                  False
location                True
department              True
salary_range            True
company_profile         True
description             True
requirements            True
benefits                True
telecommuting          False
has_company_logo       False
has_questions          False
employment_type         True
required_experience     True
required_education      True
industry                True
function                True
fraudulent             False
dtype: bool

In [ ]:
# check for num of na values
df.isna().sum()

job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [ ]:
# handle location and description first since easier

# handle location - split into nation and city and fillna with unknown
df['location'] = df['location'].fillna('NA, Unknown')
df['country'] = df['location'].apply(lambda x : x.strip()[:2])
df['city'] = df['location'].apply(lambda x : x.split(',')[-1])

# a lot of sparse values noted for countries, may lead to unexpected results - to handle - keep countries with counts >10 only and put others for the rest
ls_of_countries = [country if df[df['country']==country]['country'].count() >= 10 else 'Others' for country in df['country'].unique()]
df['country'] = df['country'].apply(lambda x : x if x in ls_of_countries else 'Others')

ls_of_cities = [city if df[df['city']==city]['city'].count() >= 10 else 'Others' for city in df['city'].unique()]
df['city'] = df['city'].apply(lambda x : x if x in ls_of_cities else 'Others')

# keep track in cols_to_keep and cols_to_drop
cols_to_keep.append('country')
cols_to_keep.append('city')
cols_to_drop.append('location')

In [ ]:
# handle description - convert to binary - with or without
df['has_description'] = df['description'].apply(lambda x: 0 if pd.isna(x) else 1)

# keep track in cols_to_keep and cols_to_drop
cols_to_keep.append('has_description')
cols_to_drop.append('description')

In [ ]:
# handle department - q a lot of departments - keep only those with high count and take everything else as others
ls_of_dept = [dept if df[df['department']==dept]['department'].count() >= 10 else 'Others' for dept in df['department'].unique()]
df['has_department'] = df['department'].apply(lambda x : x if x in ls_of_dept else 'Others')

# keep track in cols_to_keep and cols_to_drop
cols_to_keep.append('has_department')
cols_to_drop.append('department')

In [ ]:
# handle salary_range - q a lot of ranges - keep as binary - has_salary or not
df['has_salary'] = df['salary_range'].apply(lambda x : 0 if pd.isna(x) else 1)

# keep track in cols_to_keep and cols_to_drop
cols_to_keep.append('has_salary')
cols_to_drop.append('salary_range')

In [ ]:
cols_to_drop # handled these colummns

['location', 'description', 'department', 'salary_range']

In [ ]:
# handle company_profile, requirements and benefits next - for simplicity - keep as binary - has or not
df['has_company_profile'] = df['company_profile'].apply(lambda x : 0 if pd.isna(x) else 1)
df['has_requirements'] = df['requirements'].apply(lambda x : 0 if pd.isna(x) else 1)
df['has_benefits'] = df['benefits'].apply(lambda x : 0 if pd.isna(x) else 1)

# keep track in cols_to_keep and cols_to_drop
cols_to_keep.append('has_company_profile')
cols_to_keep.append('has_requirements')
cols_to_keep.append('has_benefits')
cols_to_drop.append('company_profile')
cols_to_drop.append('requirements')
cols_to_drop.append('benefits')

In [ ]:
# handle employment_type - keep na values as unknown
df['employment_type'] = df['employment_type'].fillna('Unknown')

# keep track in cols_to_keep and cols_to_drop
cols_to_keep.append('employment_type')

In [ ]:
df['required_experience'].unique()

array(['Internship', 'Not Applicable', nan, 'Mid-Senior level',
       'Associate', 'Entry level', 'Executive', 'Director'], dtype=object)

In [ ]:
# handle required_experience - keep na values as unknown
df['required_experience'] = df['required_experience'].fillna('Unknown')

# keep track in cols_to_keep and cols_to_drop
cols_to_keep.append('required_experience')

In [ ]:
df['required_education'].unique()

array([nan, "Bachelor's Degree", "Master's Degree",
       'High School or equivalent', 'Unspecified',
       'Some College Coursework Completed', 'Vocational', 'Certification',
       'Associate Degree', 'Professional', 'Doctorate',
       'Some High School Coursework', 'Vocational - Degree',
       'Vocational - HS Diploma'], dtype=object)

In [ ]:
# handle required_education - keep na values as unknown
df['required_education'] = df['required_education'].fillna('Unknown')

# keep track in cols_to_keep and cols_to_drop
cols_to_keep.append('required_education')

In [ ]:
cols_to_keep

['country',
 'city',
 'has_description',
 'has_department',
 'has_salary',
 'has_company_profile',
 'has_requirements',
 'has_benefits',
 'employment_type',
 'required_experience',
 'required_education']

In [ ]:
# handle industry - keep the industries - fillna with unknown
df['industry'] = df['industry'].fillna('Unknown')

# a lot of sparse values noted for industries, may lead to unexpected results - to handle - keep countries with counts >10 only and put unknown for the rest
ls_of_industries = [industry if df[df['industry']==industry]['industry'].count() >= 10 else 'Others' for industry in df['industry'].unique()]
df['industry'] = df['industry'].apply(lambda x : x if x in ls_of_industries else 'Others')

# keep track in cols_to_keep and cols_to_drop
cols_to_keep.append('industry')

In [ ]:
# handle function - keep the functions - fillna with unknown
df['function'] = df['function'].fillna('Unknown')

# keep track in cols_to_keep and cols_to_drop
cols_to_keep.append('function')

In [ ]:
cols_to_keep

['country',
 'city',
 'has_description',
 'has_department',
 'has_salary',
 'has_company_profile',
 'has_requirements',
 'has_benefits',
 'employment_type',
 'required_experience',
 'required_education',
 'industry',
 'function']

In [ ]:
cols_to_drop

['location',
 'description',
 'department',
 'salary_range',
 'company_profile',
 'requirements',
 'benefits']

## Next we look into the non null columns to extract what we wanna keep

In [ ]:
# drop the cols inside cols_to_drop as it is not useful anymore
df = df.drop(cols_to_drop, axis=1)

In [ ]:
cols_to_drop = []

In [ ]:
df.isna().sum()

job_id                 0
title                  0
telecommuting          0
has_company_logo       0
has_questions          0
employment_type        0
required_experience    0
required_education     0
industry               0
function               0
fraudulent             0
country                0
city                   0
has_description        0
has_department         0
has_salary             0
has_company_profile    0
has_requirements       0
has_benefits           0
dtype: int64

In [ ]:
# handle job_id - it is unique for all - so not much value - remove the col
df = df.drop('job_id', axis=1)

In [ ]:
# handle title - a bit too many distinct roles - may lead to overfitting issues - would remove the cols - flexible to choose to ohe or use as text feature
df = df.drop('title', axis=1)

In [ ]:
# handle telecommuting, has_company_logo, has_questions - all are binary - keep as features
# keep track in cols_to_keep
cols_to_keep.append('telecommuting')
cols_to_keep.append('has_company_logo')
cols_to_keep.append('has_questions')

And with that all the columns are processed accordingly

In [ ]:
cols_to_keep

['country',
 'city',
 'has_description',
 'has_department',
 'has_salary',
 'has_company_profile',
 'has_requirements',
 'has_benefits',
 'employment_type',
 'required_experience',
 'required_education',
 'industry',
 'function',
 'telecommuting',
 'has_company_logo',
 'has_questions']

## Text Preprocessing

In [ ]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Check if the text is NaN
    if pd.isnull(text):
        return ""

    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Convert to lowercase
    text = text.lower()

    # Remove special characters, URLs, and punctuation
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    # Tokenization and removal of stop words
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join the words back into a single string
    processed_text = ' '.join(words)

    return processed_text

# List of textual columns to preprocess
textual_columns = ["title", "company_profile", "description", "requirements", "benefits"]

# Apply the preprocessing function to all specified columns in 'data'
for column in textual_columns:
    data[column] = data[column].apply(preprocess_text)

# Concatenate processed text columns into a single column
data['combined_text'] = data[textual_columns].apply(lambda row: ' '.join(row), axis=1)

# Append the preprocessed text columns to 'df'
df = pd.concat([df, data['combined_text']], axis=1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
<ipython-input-32-139f3c286ca9>:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


## Train-test split and preprocessing before inputting into model

In [ ]:
# seperate out the correctly formatted cols and those which still needs processing through ohe
binary_cols = ['has_description', 'has_salary', 'has_company_profile', 'has_requirements', 'has_benefits', 'telecommuting', 'has_company_logo', 'has_questions']
ohe_needed_cols = []

for cols in cols_to_keep:
  if cols not in binary_cols:
    ohe_needed_cols.append(cols)

In [ ]:
# select the fraudulent column as target, rest as features
features = df.drop(['fraudulent', 'combined_text'], axis=1)
target_var = df['fraudulent']

In [ ]:
# do ohe for ohe_needed_cols
features_encoded = pd.get_dummies(features, columns=ohe_needed_cols, prefix=ohe_needed_cols).astype(int)

##Count Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
vectorizer = CountVectorizer(max_features=5000) #top 5000 most frequent words

# Fit and transform the combined_text column
text_features = vectorizer.fit_transform(df['combined_text'])

# Create a DataFrame with the count vectorized features
text_df = pd.DataFrame(text_features.toarray(), columns=vectorizer.get_feature_names_out())

# Concatenate the text features DataFrame with the original 'df'
features_encoded = pd.concat([features_encoded, text_df], axis=1)


In [ ]:
features_encoded # check the colummns if its in correct format

,telecommuting,has_company_logo,has_questions,has_description,has_salary,has_company_profile,has_requirements,has_benefits,country_AE,country_AT,...,youth,youtube,yr,zealand,zend,zendesk,zero,zijn,zone,zylun
0,0,1,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,1,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,1,0,1,1,1,0,0,...,0,0,0,0,0,0,2,0,0,0
4,0,1,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,0,1,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
17876,0,1,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
17877,0,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17878,0,0,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Identify duplicate columns
duplicate_columns = features_encoded.columns[features_encoded.columns.duplicated()]

# Print duplicate columns
print("Duplicate Columns:", duplicate_columns)

# Drop duplicate columns
features_encoded = features_encoded.loc[:, ~features_encoded.columns.duplicated()]


Duplicate Columns: Index([], dtype='object')


In [ ]:
# Check for duplicate column names
duplicate_columns = features_encoded.columns[features_encoded.columns.duplicated()]
print("Duplicate Columns:", duplicate_columns)


Duplicate Columns: Index([], dtype='object')


In [ ]:
# train test split
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(features_encoded, target_var, random_state=42, test_size=0.2)

## Model Evaluation

In [ ]:
# model packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report, roc_auc_score
from xgboost import XGBClassifier

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore FutureWarnings and DeprecationWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

In [ ]:
# import the models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

# Set the random seed for reproducibility
np.random.seed(42)

# perform model training and model comparisons
list_of_models = [LogisticRegression() , KNeighborsClassifier() ,
                  DecisionTreeClassifier() ,ExtraTreeClassifier(), RandomForestClassifier(), ExtraTreesClassifier(),
                  SVC(kernel='rbf',probability=True), AdaBoostClassifier(), XGBClassifier()]
classification_report = pd.DataFrame(columns=['Accuracy','F1_score','Precision','Recall'])

for model in list_of_models :
    if isinstance(model, SVC):
        # Standardize the features only for SVC
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(xtrain)
        X_test_scaled = scaler.transform(xtest)
        model.fit(X_train_scaled, ytrain)
    else:
        # For other models, use unscaled features
        model.fit(xtrain, ytrain)

    for i in range(2):
        if i == 0:
            to_pred = X_train_scaled if isinstance(model, SVC) else xtrain
            pred = ytrain
            title = 'Train'
        else:
            to_pred = X_test_scaled if isinstance(model, SVC) else xtest
            pred = ytest
            title = 'Test'
        model_name = str(type(model)).split(".")[-1][:-2]
        y_pred = model.predict(to_pred)
        y_probs = model.predict_proba(to_pred)[:, 1]
        acc = round(accuracy_score(pred, y_pred)*100)
        f1 = round(f1_score(pred, y_pred)*100)
        prec = round(precision_score(pred, y_pred)*100)
        recall = round(recall_score(pred, y_pred)*100)
        rocauc =  round(roc_auc_score(pred, y_probs)*100)
        d = pd.DataFrame(data=np.array([acc,f1,prec,recall,rocauc]).reshape(1,5)
                     , columns=['Accuracy' , 'F1_score' , 'Precision' , 'Recall', 'Roc Auc'])
        classification_report = pd.concat([classification_report , d])
        classification_report.rename( index= { 0 :'{} _ {} Details'.format(model_name , title) } ,inplace=True )
pd.options.display.max_rows = None
classification_report

,Accuracy,F1_score,Precision,Recall,Roc Auc
LogisticRegression _ Train Details,100,100,100,100,100.0
LogisticRegression _ Test Details,98,82,90,75,97.0
KNeighborsClassifier _ Train Details,97,70,60,83,99.0
KNeighborsClassifier _ Test Details,94,56,45,73,93.0
DecisionTreeClassifier _ Train Details,100,100,100,100,100.0
DecisionTreeClassifier _ Test Details,98,78,76,81,90.0
ExtraTreeClassifier _ Train Details,100,100,100,100,100.0
ExtraTreeClassifier _ Test Details,96,62,62,63,80.0
RandomForestClassifier _ Train Details,100,100,100,100,100.0
RandomForestClassifier _ Test Details,98,76,100,62,99.0


## Use SMOTE for oversampling

In [ ]:
target_var.value_counts()

0    17014
1      866
Name: fraudulent, dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTENC # smotenc is used as it can handle categorical variable
from imblearn.over_sampling import SMOTE

# oversampling
smote = SMOTE(random_state=42)
xtrain_resampled, ytrain_resampled = smote.fit_resample(xtrain, ytrain)

In [ ]:
# import the models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

# Set the random seed for reproducibility
np.random.seed(42)

# perform model training and model comparisons
list_of_models = [LogisticRegression() , KNeighborsClassifier() ,
                  DecisionTreeClassifier() ,ExtraTreeClassifier(), RandomForestClassifier(), ExtraTreesClassifier(),
                  SVC(kernel='rbf',probability=True), AdaBoostClassifier(), XGBClassifier()]
classification_report = pd.DataFrame(columns=['Accuracy','F1_score','Precision','Recall'])

for model in list_of_models :
    if isinstance(model, SVC):
        # For SVC, scale the features
        scaler = StandardScaler()
        xtrain_scaled = scaler.fit_transform(xtrain_resampled)
        xtest_scaled = scaler.transform(xtest)
        model = model.fit(xtrain_scaled, ytrain_resampled)
    else:
        # For other models, use unscaled features
        model = model.fit(xtrain_resampled, ytrain_resampled)

    for i in range(2) :
        if i == 0:
            to_pred = xtrain_scaled if isinstance(model, SVC) else xtrain_resampled
            pred = ytrain_resampled
            title = 'Train'
        else:
            to_pred = xtest_scaled if isinstance(model, SVC) else xtest
            pred = ytest
            title = 'Test'
        model_name = str(type(model)).split(".")[-1][:-2]
        y_pred = model.predict(to_pred)
        y_probs = model.predict_proba(to_pred)[:, 1]
        acc = round(accuracy_score(pred, y_pred)*100)
        f1 = round(f1_score(pred, y_pred)*100)
        prec = round(precision_score(pred, y_pred)*100)
        recall = round(recall_score(pred, y_pred)*100)
        rocauc =  round(roc_auc_score(pred, y_probs)*100)
        d = pd.DataFrame(data=np.array([acc,f1,prec,recall,rocauc]).reshape(1,5)
                     , columns=['Accuracy' , 'F1_score' , 'Precision' , 'Recall', 'Roc Auc'])
        classification_report = pd.concat([classification_report , d])
        classification_report.rename( index= { 0 :'{} _ {} Details'.format(model_name , title) } ,inplace=True )
pd.options.display.max_rows = None
classification_report

,Accuracy,F1_score,Precision,Recall,Roc Auc
LogisticRegression _ Train Details,100,100,100,100,100.0
LogisticRegression _ Test Details,98,81,85,78,96.0
KNeighborsClassifier _ Train Details,90,91,83,100,100.0
KNeighborsClassifier _ Test Details,75,28,16,96,92.0
DecisionTreeClassifier _ Train Details,100,100,100,100,100.0
DecisionTreeClassifier _ Test Details,97,72,66,79,88.0
ExtraTreeClassifier _ Train Details,100,100,100,100,100.0
ExtraTreeClassifier _ Test Details,96,65,59,72,85.0
RandomForestClassifier _ Train Details,100,100,100,100,100.0
RandomForestClassifier _ Test Details,98,78,89,69,99.0


# LSTM Model (Before SMOTE)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.python.ops.math_ops import reduce_prod
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding,Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras import Input

numeric_cols = xtrain.columns

metrics_list = [
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.AUC(name='auc')]

def build_bilstm():

  numeric_inputs = Input(shape=(len(numeric_cols),), name='numeric_inputs')
  num_inp=layers.Dense(128, activation='relu', name='dense_num_1',
                kernel_initializer=tf.keras.initializers.he_normal(seed=42))(numeric_inputs)
  num_inp=layers.Dropout(0.1)(num_inp)
  num_inp=layers.Dense(128, activation='relu', name='dense_num_2',
                kernel_initializer=tf.keras.initializers.he_normal(seed=42))(num_inp)
  inputs = keras.layers.concatenate([num_inp])
  x = layers.Dense(32, activation='relu', name='dense_1')(inputs)
  output=layers.Dense(1, activation='sigmoid', name='sigmoid_output')(x)

  bilstm_model = Model(inputs=[numeric_inputs], outputs=[output])
  return bilstm_model

bilstm_model = build_bilstm()
print(bilstm_model.summary())

bilstm_model.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics = ['accuracy','AUC'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 numeric_inputs (InputLayer  [(None, 5546)]            0         
 )                                                               
                                                                 
 dense_num_1 (Dense)         (None, 128)               710016    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_num_2 (Dense)         (None, 128)               16512     
                                                                 
 concatenate_1 (Concatenate  (None, 128)               0         
 )                                                               
                                                                 
 dense_1 (Dense)             (None, 32)                4128  

In [ ]:
history = bilstm_model.fit([xtrain],ytrain, epochs=25, batch_size = 24, verbose = 1)

Epoch 1/25
596/596 [==============================] - 17s 6ms/step - loss: 0.0911 - accuracy: 0.9708 - auc: 0.9427
Epoch 2/25
596/596 [==============================] - 4s 7ms/step - loss: 0.0223 - accuracy: 0.9920 - auc: 0.9973
Epoch 3/25
596/596 [==============================] - 5s 9ms/step - loss: 0.0074 - accuracy: 0.9972 - auc: 0.9991
Epoch 4/25
596/596 [==============================] - 4s 7ms/step - loss: 0.0075 - accuracy: 0.9985 - auc: 0.9983
Epoch 5/25
596/596 [==============================] - 4s 6ms/step - loss: 0.0037 - accuracy: 0.9990 - auc: 0.9999
Epoch 6/25
596/596 [==============================] - 3s 5ms/step - loss: 0.0022 - accuracy: 0.9993 - auc: 0.9993
Epoch 7/25
596/596 [==============================] - 3s 6ms/step - loss: 0.0018 - accuracy: 0.9995 - auc: 0.9992
Epoch 8/25
596/596 [==============================] - 5s 9ms/step - loss: 0.0020 - accuracy: 0.9994 - auc: 0.9999
Epoch 9/25
596/596 [==============================] - 4s 7ms/step - loss: 0.0034 - accu

In [ ]:
y_pred = bilstm_model.predict([xtest])
y_pred=np.where(y_pred>0.5,1,0)

112/112 [==============================] - 0s 3ms/step


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ytest,y_pred)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def getMetrics(y_test,y_pred):
    accuracy = np.round(accuracy_score(ytest, y_pred),4)
    f1 = np.round(f1_score(ytest, y_pred),4)
    precision = np.round(precision_score(ytest, y_pred),4)
    recall = np.round(recall_score(ytest, y_pred),4)
    auroc = np.round(roc_auc_score(ytest, y_pred),4)
    return accuracy,f1,precision,recall,auroc

df_metric_list = pd.DataFrame([getMetrics(ytest,y_pred)])
df_metric_list.columns = ['accuracy','f1_score','precision','recall','auroc']
df_metric_list

,accuracy,f1_score,precision,recall,auroc
0,0.9877,0.8659,0.966,0.7845,0.8915


# LSTM (After SMOTE)

In [ ]:
history = bilstm_model.fit([xtrain_resampled],ytrain_resampled, epochs=25, batch_size = 24, verbose = 1)

Epoch 1/25
1135/1135 [==============================] - 9s 7ms/step - loss: 0.0033 - accuracy: 0.9991 - auc: 0.9999
Epoch 2/25
1135/1135 [==============================] - 9s 8ms/step - loss: 0.0010 - accuracy: 0.9999 - auc: 1.0000
Epoch 3/25
1135/1135 [==============================] - 5s 5ms/step - loss: 5.0586e-04 - accuracy: 1.0000 - auc: 1.0000
Epoch 4/25
1135/1135 [==============================] - 7s 6ms/step - loss: 2.8689e-06 - accuracy: 1.0000 - auc: 1.0000
Epoch 5/25
1135/1135 [==============================] - 5s 5ms/step - loss: 1.7983e-04 - accuracy: 0.9999 - auc: 1.0000
Epoch 6/25
1135/1135 [==============================] - 6s 5ms/step - loss: 0.0035 - accuracy: 0.9998 - auc: 0.9999
Epoch 7/25
1135/1135 [==============================] - 6s 5ms/step - loss: 0.0017 - accuracy: 0.9999 - auc: 1.0000
Epoch 8/25
1135/1135 [==============================] - 5s 5ms/step - loss: 6.8564e-05 - accuracy: 1.0000 - auc: 1.0000
Epoch 9/25
1135/1135 [==============================] - 

In [ ]:
y_pred = bilstm_model.predict([xtest])
y_pred=np.where(y_pred>0.5,1,0)

112/112 [==============================] - 0s 3ms/step


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ytest,y_pred)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def getMetrics(y_test,y_pred):
    accuracy = np.round(accuracy_score(ytest, y_pred),4)
    f1 = np.round(f1_score(ytest, y_pred),4)
    precision = np.round(precision_score(ytest, y_pred),4)
    recall = np.round(recall_score(ytest, y_pred),4)
    auroc = np.round(roc_auc_score(ytest, y_pred),4)
    return accuracy,f1,precision,recall,auroc

df_metric_list = pd.DataFrame([getMetrics(ytest,y_pred)])
df_metric_list.columns = ['accuracy','f1_score','precision','recall','auroc']
df_metric_list

,accuracy,f1_score,precision,recall,auroc
0,0.9846,0.8318,0.9315,0.7514,0.8742
